In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import h5py
import numpy as np
from typing import List, Tuple
from pathlib import Path
import lightgbm as lgb
import pickle

from alber.forecasting import get_nec_features_vitrine, load_model, create_result_dataset

# 0. Parameters

In [2]:
data_path = Path('../../Storage/alber/data.h5')
score_path = Path('../../Storage/alber/forecast.h5')

list_features = [
    'stoch_k_price_21_1',
    'wap_balance',
    'volume_imbalance',
    'rel_order_count_1_80',
    'rel_price_5_10',
    'log_return_mean_price',
    'bid_spread',
    'ask_spread',
    'Money',
    'rel_stoch_price_21_1_3',
    'rel_volume_ask_bid1',
    'bid_ask_spread2',
    'total_volume',
    'rel_price_1_5',
    'rel_price_40_80'
]

# 1. Get features vitrine

In [ ]:
%%time
train_vitrine = get_nec_features_vitrine(data_path)
train_vitrine.shape

/share/pchuykov/alber/alber/load_data.py:59: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left, 2 on the right)
  trades = pd.merge(trades, sum_w, on=['time'])
/share/pchuykov/alber/alber/load_data.py:59: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  trades = pd.merge(trades, sum_w, on=['time'])


ob.shape == (9871642, 9), trades.shape == (12522917, 4)


In [ ]:
train_vitrine

# 2. Get model, make predictions and save result

In [ ]:
model = load_model('model_mf_15f', Path('../saved_models/'))
model

In [ ]:
%%time
train_vitrine['score'] = model.predict(train_vitrine[list_features])

In [ ]:
train_vitrine['time'].shape

In [ ]:
%%time
create_result_dataset(score_path, train_vitrine['score'], train_vitrine['time'])

In [26]:
with h5py.File("../../Storage/alber/result.h5", "r") as f:
    # Print all root level object names (aka keys) 
    # these can be group or dataset names 
    print("Keys: %s" % f.keys())
    # get first object name/key; may or may NOT be a group
    a_group_key = list(f.keys())[0]
    
    print(type(f[a_group_key])) 
    
    data = list(f[a_group_key])
    print(data)
    print(f['Return']['Res'])

Keys: <KeysViewHDF5 ['Return']>
<class 'h5py._hl.group.Group'>
['Res', 'TS']
<HDF5 dataset "Res": shape (9871642,), type "<f8">


In [16]:
with h5py.File("../../Storage/alber/score.h5", "r") as f:
    # Print all root level object names (aka keys) 
    # these can be group or dataset names 
    print("Keys: %s" % f.keys())
    # get first object name/key; may or may NOT be a group
    a_group_key = list(f.keys())[0]
    
    print(type(f[a_group_key])) 
    
    data = list(f[a_group_key])
    print(data)
    print(f['Return']['Score'])

Keys: <KeysViewHDF5 ['Return']>
<class 'h5py._hl.group.Group'>
['Score', 'TS']
<HDF5 dataset "Score": shape (1000,), type "<f8">
